In [1]:
pip install cloudscraper

     |████████████████████████████████| 102kB 4.8MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 


In [2]:
pip install pyomo

     |████████████████████████████████| 9.5MB 7.3MB/s 
     |████████████████████████████████| 256kB 35.4MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 163kB 34.0MB/s 


In [3]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 160975 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [4]:
import requests
from bs4 import BeautifulSoup
import cloudscraper
from pyomo.environ import *

In [5]:
class Weapon:
  def __init__(self):
    self.name = ""
    self.accuracy = 0
    self.range = 0
    self.mobility = 0
    self.damage = 0
    self.firerate = 0
    self.control = 0

    self.plus_accuracy = 0
    self.plus_range = 0
    self.plus_mobility = 0
    self.plus_damage = 0
    self.plus_firerate = 0
    self.plus_control = 0

    self.attachments = list()

  def __str__(self):
    ret = "Name: {0}\n\tAccuracy: {1:.2f} (+{2})\n\tRange: {3:.2f} (+{4})\n\tMobility: {5:.2f} (+{6})\n\tDamage: {7:.2f} (+{8})\n\tFire Rate: {9:.2f} (+{10})\n\tControl: {11:.2f} (+{12})\n\t".format(self.name,
                                             self.accuracy+self.plus_accuracy,
                                             self.plus_accuracy,
                                             self.range+self.plus_range,
                                             self.plus_range,
                                             self.mobility+self.plus_mobility,
                                             self.plus_mobility,
                                             self.damage+self.plus_damage,
                                             self.plus_damage,
                                             self.firerate+self.plus_firerate,
                                             self.plus_firerate,
                                             self.control+self.plus_control,
                                             self.plus_control)
    if len(self.attachments) > 0:
      ret += "\n"
      for attachment in self.attachments:
        ret += attachment.name + "\n"

    return ret

  def attach(self, attachment):
    self.attachments.append(attachment)
    self.plus_accuracy += attachment.accuracy
    self.plus_range += attachment.range
    self.plus_mobility += attachment.mobility
    self.plus_damage += attachment.damage
    self.plus_firerate += attachment.firerate
    self.plus_control += attachment.control

  def clearattachments(self):
    self.attachments.clear()
    self.plus_accuracy = 0
    self.plus_range = 0
    self.plus_mobility = 0
    self.plus_damage = 0
    self.plus_firerate = 0
    self.plus_control = 0

class Attachment:
  def __init__(self, nam, typ, acc, ran, mob, dam, fir, con):
    self.name = nam
    self.type = typ
    self.accuracy = acc
    self.range = ran
    self.mobility = mob
    self.damage = dam
    self.firerate = fir
    self.control = con
    

  


# Start Scrapping URL

In [6]:
url = "http://cod.tracker.gg"
sub_url = "/warzone/db/weapons"

scraper = cloudscraper.create_scraper()

#page = requests.get(url + sub_url)
page = scraper.get(url + sub_url)

soup = BeautifulSoup(page.text, 'html.parser')

pages = 7
weapons = list()
for page_id in range(1,pages+1):
  page = scraper.get(url + sub_url + "?page=" + str(page_id))
  soup = BeautifulSoup(page.text, 'html.parser')
  divTag = soup.findAll("a", {"data-type": "Weapon"}, href = True)
  for tag in divTag:
    weapon_url = tag['href']
    print("Loading data for " + weapon_url)
    weapon_page = scraper.get(url + weapon_url)
    soup2 = BeautifulSoup(weapon_page.text, 'html.parser')

    weapon = Weapon()

    # Load weapon name and details
    weapon.name = soup2.findAll("h1", {"class": "weapon-header__title"})[0].text
    for tag2 in soup2.findAll("div", {"class": "stat"}):
      data = tag2.text.rstrip()
      data_lst = data.split(" ")
      value = float(data_lst[-1])
      attr_name = ""
      for i in range(len(data_lst)-1):
        attr_name += data_lst[i].lower()
      setattr(weapon, attr_name, value)

    # Load weapon attachments
    attachments = list()
    
    weapons.append(weapon)

print("Loaded {} weapons".format(len(weapons)))

Loading data for /warzone/db/weapons/5-357
Loading data for /warzone/db/weapons/3-50-gs
Loading data for /warzone/db/weapons/33207244-1911
Loading data for /warzone/db/weapons/4-1911
Loading data for /warzone/db/weapons/36-725
Loading data for /warzone/db/weapons/31370411-ak-47
Loading data for /warzone/db/weapons/16-ak-47
Loading data for /warzone/db/weapons/30134784-ak-74u
Loading data for /warzone/db/weapons/807-an-94
Loading data for /warzone/db/weapons/994-as-val
Loading data for /warzone/db/weapons/8-aug
Loading data for /warzone/db/weapons/27358830-aug
Loading data for /warzone/db/weapons/30-ax-50
Loading data for /warzone/db/weapons/546-bruen-mk9
Loading data for /warzone/db/weapons/33149760-bullfrog
Loading data for /warzone/db/weapons/626-cr-56-amax
Loading data for /warzone/db/weapons/1122-cx-9
Loading data for /warzone/db/weapons/22685878-cigma-2
Loading data for /warzone/db/weapons/14-combat-knife
Loading data for /warzone/db/weapons/33-crossbow
Loading data for /warzone/d

# Define attachments

In [7]:
attachments_dict = {
    "XM4": [
            # 71 63 63 77 70 79
            Attachment("1mW Laser", "Laser", 2, 0, 1, 0, 0, -1),
            Attachment("5mW Laser", "Laser", 4, 0, 2, 0, 0, -1),
            Attachment("Tac Laser", "Laser", 2, 0, 2, 0, 0, 3),
            Attachment("Tac Laser 2", "Laser", 2, 0, 2, 0, 0, 3),
            Attachment("Compensator", "Muzzle", 2, 0, -1, 0, 0, 3),
            Attachment("Tactical Supressor", "Muzzle", 1.5, 0, -1.5, 0, 0, -0.5),
            Attachment("Reflex", "Optic", 1, 0, -1, 0, 0, -1),
            Attachment("Scout Combat Optic", "Optic", 3, 4, -3, 0, 0, -3),
            Attachment("Variable Zoom Scope", "Optic", 1, 4, -3, 0, 0, -2),
            Attachment("Granulated Grip Tape", "Rear Grip", 2, 0, -2, 0, 0, 2),
            Attachment("Rubberized Grip Tape", "Rear Grip", 4, 0, 0, 0, 0, -2),
            Attachment("Stippled Grip Tape", "Rear Grip", 0, 0, 3, 0, 0, -2),
            Attachment("Commando Foregrip", "Underbarrel", 4, 2, -2, 0, 0, 3),
            Attachment("Merc Foregrip", "Underbarrel", 5, 1, -1.5, 0, 0, 2),
            Attachment("Operator Foregrip", "Underbarrel", 4, 1.5, -1, 0, 0, 1),
            Attachment("Tactical Foregrip", "Underbarrel", 1, 1, -1, 0, 0, 6)
    ],
    "M4A1": [
             #71 63 63 72  75 71
             Attachment("50 Round Mags", "Ammunition", 0, 0, -4, 0, 0, -4),
             Attachment("60 Round Mags", "Ammunition", 0, 0, -4, 0, 0, -2.5),
             Attachment("9mm Para 32-Round Mags", "Ammunition", 4, -7, 2, -8, 16, 6),
             Attachment("Corvus Custom Marksman", "Barrel", 3, 5, -3, 0, 0, 4),
             Attachment("FSS 11.5 Commando", "Barrel", -2, -4, 6, 0, 0, -2),
             Attachment("FSS 12.4 Predator", "Barrel", 2, -6, 3, 0, 0, 0),
             Attachment("FSS 14.5 Tac Lite", "Barrel", 0, -3, 3, 0, 0, 0),
             Attachment("1mW Laser", "Laser", 2, 0, 1, 0, 0, -1),
             Attachment("5mW Laser", "Laser", 4, 0, 2, 0, 0, -1),
             Attachment("Tac Laser", "Laser", 4, 0, 4, 0, 0, 6),
             Attachment("Compensator", "Muzzle", 4, 0, -2, 0, 0, 6),
             Attachment("Flash Guard", "Muzzle", 0, -1, -0.5, 0, 0, 0),
             Attachment("Muzzle Brake", "Muzzle", 7, 0, -2, 0, 0, 2),
             Attachment("Tactical Supressor", "Muzzle", 1.5, 0, -1.5, 0, 0, -0.5),
             Attachment("Integral Hybrid", "Optic", 4, 5, -2, 0, 0, -3),
             Attachment("Reflex", "Optic", 1, 0, -1, 0, 0, -1),
             Attachment("Scout Combat Optic", "Optic", 5, 4, -3, 0, 0, -3),
             Attachment("Solozero NVG Enhanced", "Optic", 5, 4, -4, 0, 0, -4),
             Attachment("Variable Zoom Scope", "Optic", 1, 4, -3, 0, 0, -2),
             Attachment("Granulated Grip Tape", "Rear Grip", 4, 0, -4, 0, 0, 4),
             Attachment("Rubberized Grip Tape", "Rear Grip", 8, 0, 0, 0, 0, -4),
             Attachment("Strippled Grip Tape", "Rear Grip", 0, 0, 6, 0, 0, -4),
             Attachment("FORGE TAC CQS", "Stock", -2, 0, 3, 0, 0, -1),
             Attachment("M-16 Stock", "Stock", 2, 0, -2, 0, 0, 3),
             Attachment("No Stock", "Stock", -3, -2, 9, 0, 0, -4),
             Attachment("Singuard Arms Invader", "Stock", -1, 0, 4, 0, 0, -2.5),
             Attachment("Bipod", "Underbarrel", -1, 0, 4, 0, 0, -2.5),
             Attachment("Commando Foregrip", "Underbarrel", 8, 4, -4, 0, 0, 6),
             Attachment("Merc Foregrip", "Underbarrel", 10, 2, -3, 0, 0, 4),
             Attachment("Operator Foregrip", "Underbarrel", 8, 3, -2, 0, 0, 2),
             Attachment("Tactical Foregrip", "Underbarrel", 1, 1, -1, 0, 0, 6)
    ]
}

### Define Weapon to optimize

In [8]:
WEAPON = "M4A1"
weapon = None
for w in weapons:
  if w.name == WEAPON:
    weapon = w
    break
print(weapon)

Name: M4A1
	Accuracy: 71.00 (+0)
	Range: 63.00 (+0)
	Mobility: 63.00 (+0)
	Damage: 72.00 (+0)
	Fire Rate: 75.00 (+0)
	Control: 71.00 (+0)
	


# Create Optimization Model

First, count number of lasers, optics, reargrips, underbarrels and muzzles for this weapon

In [9]:
n_laser = len([x for x in attachments_dict[weapon.name] if x.type == "Laser"])
n_optic = len([x for x in attachments_dict[weapon.name] if x.type == "Optic"])
n_rgrip = len([x for x in attachments_dict[weapon.name] if x.type == "Rear Grip"])
n_ubarrel = len([x for x in attachments_dict[weapon.name] if x.type == "Underbarrel"])
n_muzzle = len([x for x in attachments_dict[weapon.name] if x.type == "Muzzle"])

n_ammo = len([x for x in attachments_dict[weapon.name] if x.type == "Ammunition"])
n_barrel = len([x for x in attachments_dict[weapon.name] if x.type == "Barrel"])
n_stock = len([x for x in attachments_dict[weapon.name] if x.type == "Stock"])


In [10]:
model = ConcreteModel()

model.Lasers = Set(initialize = range(n_laser))
model.Optics = Set(initialize = range(n_optic))
model.Reargrips = Set(initialize = range(n_rgrip))
model.Underbarrels = Set(initialize = range(n_ubarrel))
model.Muzzles = Set(initialize = range(n_muzzle))
model.Ammunitions = Set(initialize = range(n_ammo))
model.Barrels = Set(initialize = range(n_barrel))
model.Stocks = Set(initialize = range(n_stock))

model.laser = Var(model.Lasers, within = Binary)
model.optic = Var(model.Optics, within = Binary)
model.reargrip = Var(model.Reargrips, within = Binary)
model.underbarrel = Var(model.Underbarrels, within = Binary)
model.muzzle = Var(model.Muzzles, within = Binary)
model.ammunition = Var(model.Ammunitions, within = Binary)
model.barrel = Var(model.Barrels, within = Binary)
model.stock = Var(model.Stocks, within = Binary)

### Objective Function

In [11]:
def ObjectiveFunction(model):

  lasers = [x for x in attachments_dict[weapon.name] if x.type == "Laser"]
  optics = [x for x in attachments_dict[weapon.name] if x.type == "Optic"]
  reargrips = [x for x in attachments_dict[weapon.name] if x.type == "Rear Grip"]
  ubarrels = [x for x in attachments_dict[weapon.name] if x.type == "Underbarrel"]
  muzzles = [x for x in attachments_dict[weapon.name] if x.type == "Muzzle"]
  ammunitions = [x for x in attachments_dict[weapon.name] if x.type == "Ammunition"]
  barrels = [x for x in attachments_dict[weapon.name] if x.type == "Barrel"]
  stocks = [x for x in attachments_dict[weapon.name] if x.type == "Stock"]

  ATTRS_TO_OPTIMIZE = {
      "accuracy": 0.0,
      "range": 0.0,
      "mobility": 0.0,
      "damage": 0.5,
      "firerate": 0.0,
      "control": 0.5
  }

  assert sum(ATTRS_TO_OPTIMIZE.values()) == 1.0, "Sum of weights must be equal to 1.0"

  rest_value = 0
  for variable in ATTRS_TO_OPTIMIZE:
    factor = ATTRS_TO_OPTIMIZE[variable]
    for i in model.Lasers:
      rest_value += factor*model.laser[i]*getattr(lasers[i], variable)

    for i in model.Optics:
      rest_value += factor*model.optic[i]*getattr(optics[i], variable)

    for i in model.Reargrips:
      rest_value += factor*model.reargrip[i]*getattr(reargrips[i], variable)

    for i in model.Underbarrels:
      rest_value += factor*model.underbarrel[i]*getattr(ubarrels[i], variable)

    for i in model.Muzzles:
      rest_value += factor*model.muzzle[i]*getattr(muzzles[i], variable)

    for i in model.Ammunitions:
      rest_value += factor*model.ammunition[i]*getattr(ammunitions[i], variable)

    for i in model.Barrels:
      rest_value += factor*model.barrel[i]*getattr(barrels[i], variable)

    for i in model.Stocks:
      rest_value += factor*model.stock[i]*getattr(stocks[i], variable)

  return rest_value

### Constraints

In [12]:
def NumberOfAttachments(self):
  # This constraints ensures that the number of attachments is equal to 5
  attachments = 0
  for i in model.Lasers:
    attachments += model.laser[i]

  for i in model.Optics:
    attachments += model.optic[i]
  
  for i in model.Reargrips:
    attachments += model.reargrip[i]

  for i in model.Underbarrels:
    attachments += model.underbarrel[i]

  for i in model.Muzzles:
    attachments += model.muzzle[i]

  for i in model.Ammunitions:
    attachments += model.ammunition[i]

  for i in model.Barrels:
    attachments += model.barrel[i]

  for i in model.Stocks:
    attachments += model.stock[i]

  return attachments == 5


### Build Model

In [13]:
model.obj = Objective(rule = ObjectiveFunction, sense = maximize)
model.c0 = Constraint(rule = NumberOfAttachments)
model.c1 = Constraint(expr = sum(model.laser[i] for i in model.Lasers) <= 1)
model.c2 = Constraint(rule = sum(model.optic[i] for i in model.Optics)  ==1) # setting this equal to 1 ensures that at least one optics is selected
model.c3 = Constraint(rule = sum(model.reargrip[i] for i in model.Reargrips) <= 1)
model.c4 = Constraint(rule = sum(model.underbarrel[i] for i in model.Underbarrels) <= 1)
model.c5 = Constraint(rule = sum(model.muzzle[i] for i in model.Muzzles) <= 1)
if n_ammo > 0:
  model.c6 = Constraint(rule = sum(model.ammunition[i] for i in model.Ammunitions) <= 1)

if n_barrel > 0:
  model.c7 = Constraint(rule = sum(model.barrel[i] for i in model.Barrels) <= 1)

if n_stock > 0:
  model.c8 = Constraint(rule = sum(model.stock[i] for i in model.Stocks) <= 1)

### Solve

In [14]:
solver = SolverFactory('glpk', executable='/usr/bin/glpsol')
results = solver.solve(model)

### Attach optimal attachments to weapon


In [15]:
weapon.clearattachments()
weapon.attachments.clear()
lasers = [x for x in attachments_dict[weapon.name] if x.type == "Laser"]
optics = [x for x in attachments_dict[weapon.name] if x.type == "Optic"]
reargrips = [x for x in attachments_dict[weapon.name] if x.type == "Rear Grip"]
ubarrels = [x for x in attachments_dict[weapon.name] if x.type == "Underbarrel"]
muzzles = [x for x in attachments_dict[weapon.name] if x.type == "Muzzle"]
ammunitions = [x for x in attachments_dict[weapon.name] if x.type == "Ammunition"]
barrels = [x for x in attachments_dict[weapon.name] if x.type == "Barrel"]
stocks = [x for x in attachments_dict[weapon.name] if x.type == "Stock"]
for i in model.Lasers:
  if model.laser[i]() == 1:
    weapon.attach(lasers[i])

for i in model.Optics:
  if model.optic[i]() == 1:
    weapon.attach(optics[i])

for i in model.Muzzles:
  if model.muzzle[i]() == 1:
    weapon.attach(muzzles[i])

for i in model.Reargrips:
  if model.reargrip[i]() == 1:
    weapon.attach(reargrips[i])

for i in model.Underbarrels:
  if model.underbarrel[i]() == 1:
    weapon.attach(ubarrels[i])

for i in model.Ammunitions:
  if model.ammunition[i]() == 1:
    weapon.attach(ammunitions[i])

for i in model.Barrels:
  if model.barrel[i]() == 1:
    weapon.attach(barrels[i])

for i in model.Stocks:
  if model.stock[i]() == 1:
    weapon.attach(stocks[i])

# Print results

In [16]:
print(weapon)

Name: M4A1
	Accuracy: 92.00 (+21)
	Range: 67.00 (+4)
	Mobility: 56.00 (+-7)
	Damage: 72.00 (+0)
	Fire Rate: 75.00 (+0)
	Control: 92.00 (+21)
	
Tac Laser
Reflex
Compensator
Granulated Grip Tape
Commando Foregrip



### Run the following code to see the results of the optimization. It must show 'optimal' at 'Termination Condition'. If not, the results are not accurate

In [ ]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 13.0
  Upper bound: 13.0
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 32
  Number of nonzeros: 63
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.024033308029174805
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------

In [ ]:
weapons_sorted = sorted(weapons, key = lambda x: x.control, reverse = True)

In [ ]:
print(weapons_sorted[-15])

Name: MK2 Carbine
	Accuracy: 78.00 (+0)
	Range: 68.00 (+0)
	Mobility: 66.00 (+0)
	Damage: 80.00 (+0)
	Fire Rate: 45.00 (+0)
	Control: 60.00 (+0)
	
